In [1]:
import hashlib
import time
import json

class Block:
    def __init__(self, index, previous_hash, timestamp, data, nonce=0):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.nonce = nonce
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = f"{self.index}{self.previous_hash}{self.timestamp}{json.dumps(self.data)}{self.nonce}"
        return hashlib.sha256(block_string.encode()).hexdigest()

    def mine_block(self, difficulty):
        # proof-of-work: find a hash with `difficulty` leading zeros
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()
        print(f"Block mined: {self.hash}")


class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.difficulty = 3  # adjust mining difficulty
        self.pending_transactions = []
        self.mining_reward = 10

    def create_genesis_block(self):
        return Block(0, "0", time.time(), "Genesis Block")

    def get_latest_block(self):
        return self.chain[-1]

    def mine_pending_transactions(self, miner_address):
        block = Block(len(self.chain), self.get_latest_block().hash, time.time(), self.pending_transactions)
        block.mine_block(self.difficulty)
        self.chain.append(block)

        # Reset pending transactions and reward miner
        self.pending_transactions = [
            {"from": "System", "to": miner_address, "amount": self.mining_reward}
        ]

    def create_transaction(self, transaction):
        self.pending_transactions.append(transaction)

    def get_balance(self, address):
        balance = 0
        for block in self.chain:
            if isinstance(block.data, list):  # check only tx blocks
                for tx in block.data:
                    if tx["from"] == address:
                        balance -= tx["amount"]
                    if tx["to"] == address:
                        balance += tx["amount"]
        return balance

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]

            if current.hash != current.calculate_hash():
                return False
            if current.previous_hash != previous.hash:
                return False
        return True


# ------------------------
# Testing our blockchain
# ------------------------

mycoin = Blockchain()

print("⛏️ Mining first block...")
mycoin.mine_pending_transactions("miner1")

mycoin.create_transaction({"from": "Alice", "to": "Bob", "amount": 50})
mycoin.create_transaction({"from": "Bob", "to": "Charlie", "amount": 20})

print("⛏️ Mining second block...")
mycoin.mine_pending_transactions("miner1")

print(f"Balance of miner1: {mycoin.get_balance('miner1')}")
print(f"Is blockchain valid? {mycoin.is_chain_valid()}")


⛏️ Mining first block...
Block mined: 0000709f237795b80645f3ba7f78bc8036d03c8574172921a0278758ef8d358e
⛏️ Mining second block...
Block mined: 00059622ced5c0f17ece552e9c1bd5da4b3964b1206383d7c85fc03d4e904f92
Balance of miner1: 10
Is blockchain valid? True
